In [1]:
import pandas as pd
import re
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import locationtagger

In [2]:
df = pd.read_csv("google_review_29_08_v3.csv")
df.head()

,SVAAS_ID,user_id,firstName,middleName,lastName,email,phone,description,pan,mci,...,numberFollowUp,followUpDuration,insuranceProviderId,isActive,ABDM_Healthcare_Professional_ID,SVAAS_DOCTOR_ID,svaasClinicId,tbl_user_phone_number,name,location
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['firstName', 'lastName', 'name', 'baseCity']]
df.head()

,firstName,lastName,name,baseCity
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN


In [4]:
df_not_missing = df.dropna()

In [5]:
df_not_missing.head()

,firstName,lastName,name,baseCity
17,Divya,Kavuri,Apollo Clinic,Hyderabad
18,Shivam,P,Shivam hospitals,Hyderabad
19,Ranjan,Dey,Ranjan skin clinics,Hyderabad
20,Soumya,Sarkar,Soumya Multi speciality clinic,Hyderabad
21,Rishanku,Goyal,RKG Eye Center,Hyderabad


In [6]:
df_not_missing.isna().sum()

firstName    0
lastName     0
name         0
baseCity     0
dtype: int64

In [7]:
len(df_not_missing)/len(df)

0.9879032258064516

In [8]:
df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \
                          +"+"+ df_not_missing['name'].apply(lambda x:x.lower()) + "+" + df_not_missing["baseCity"].apply(lambda x:x.lower())

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_79681/4047833984.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['firstName'].apply(lambda x:x.lower()) +"+" + df_not_missing['lastName'].apply(lambda x:x.lower()) \


In [9]:
df_not_missing['query'].values.shape

(1715,)

## For missing data

In [10]:
import preprocess_kgptalkie as ps
import re

def get_clean(x):
    x = str(x).lower().replace('\\', '').replace('_', ' ')
    x = ps.cont_exp(x)
    x = ps.remove_emails(x)
    x = ps.remove_urls(x)
    x = ps.remove_html_tags(x)
    x = ps.remove_rt(x)
    x = ps.remove_accented_chars(x)
    x = ps.remove_special_chars(x)
    x = re.sub("(.)\\1{2,}", "\\1", x)
    return x




In [11]:
df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))
df_not_missing['query'].head()

/var/folders/z1/0tn2v83n6pzgl2m8lzz1phbc0000gn/T/ipykernel_79681/153722280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_not_missing['query'] = df_not_missing['query'].apply(lambda x:ps.remove_accented_chars(x))


17                 divya+kavuri+apollo clinic+hyderabad
18                  shivam+p+shivam hospitals+hyderabad
19             ranjan+dey+ranjan skin clinics+hyderabad
20    soumya+sarkar+soumya multi speciality clinic+h...
21              rishanku+goyal+rkg eye center+hyderabad
Name: query, dtype: object

In [12]:
from collections import defaultdict

final_df = defaultdict(list)

In [13]:
df_not_missing = df_not_missing[['firstName', 'lastName', 'name', 'baseCity','query']]

In [14]:
def isDigit(char):
    pattern = "^[-+]?[0-9]*\.?[0-9]+(e[-+]?[0-9]+)?$"
    reg = re.compile(pattern)
    if reg.match(char):
        return True
    return False

In [15]:
import requests
from bs4 import BeautifulSoup
import re


    
def getLinkFromUrl(query):
    link_values = []
    page = requests.get(query)
    soup = BeautifulSoup(page.content)
    links = soup.findAll("a")
    for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
        url = re.split(":(?=http)",link["href"].replace("/url?q=",""))
        link_values.append(str(url[0]))
    return link_values 

In [16]:
links = getLinkFromUrl(query="https://google.com/search?q=shivam+p+shivam+hospitals+hyderabad")

In [17]:
for l in links:
    link = l.split("&")[0]
    print(link)

https://www.justdial.com/Hyderabad/Shivam-Clinic-Near-Hanuman-Temple-Kukatpally/040PXX40-XX40-140310200923-M5T7_BZDET
https://www.justdial.com/Hyderabad/Shivam-Clinic-Obg-Gynecology-and-Orthopedics-Near-Y-Junction-Kukatpally/040PXX40-XX40-150727214202-J1K7_BZDET
https://www.practo.com/hyderabad/clinic/shivam-clinic-kphb
https://www.drmsanjayortho.com/
https://drlogy.com/hospitals/shivam-clinic-23676
https://www.timesmed.com/Hospital/Hyderabad/shivam-clinic-115461
https://www.lybrate.com/hyderabad/clinic/shivam-clinic-kphb
https://ezz.health/hyderabad/hospitals/shivam-clinic-kukatpally
https://theshivamhospital.in/
http://www.shivamhospital.in/
https://support.google.com/websearch%3Fp%3Dws_settings_location%26hl%3Den-IN
https://accounts.google.com/ServiceLogin%3Fcontinue%3Dhttps://www.google.com/search%253Fq%253Dshivam%252Bp%252Bshivam%252Bhospitals%252Bhyderabad%26hl%3Den


In [18]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
import requests
import re

    

query="https://google.com/search?q=shivam+p+shivam+hospitals+hyderabad"

request = urllib.request.Request(query)
request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')
raw_response = urllib.request.urlopen(request).read()

html = raw_response.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

links = soup.findAll("a")
for link in  soup.find_all("a",href=re.compile("(?<=/url\?q=)(htt.*://.*)")):
    print(re.split(":(?=http)",link["href"].replace("/url?q=","")))


In [19]:
"practo" in "practo.com"

True

In [20]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   

url = "https://www.practo.com/bangalore/doctor/dr-anjanappa-t-m-general-surgeon"
request = urllib.request.Request(url)

request.add_header('User-Agent', 'your bot 0.1')
raw_response = urllib.request.urlopen(request).read()

html = raw_response.decode("utf-8")

soup = BeautifulSoup(html, 'html.parser')

heading = soup.find("h1")
speciality = soup.find("h2", class_="u-spacer--right-thin")
rating = soup.find("div", class_="u-pos-rel")
votes = soup.find("span", attrs={"data-qa-id":"clinic-votes"})
address = soup.find("p", class_="c-profile--clinic__address")

print(heading.get_text())
# print(speciality.get_text())
print(rating.get_text())
# print(votes.get_text().strip(")").strip("("))
print(address.get_text())


Dr. Anjanappa T M
4.0
1179, A Block, 17th Cross, 6th Main, 2nd Stage, Landmark: Near to Dr. Rajkumar Road, Bangalore


In [21]:
df_not_missing['query'].apply(lambda x:ps.remove_special_chars(x))

17                      divyakavuriapollo clinichyderabad
18                       shivampshivam hospitalshyderabad
19                  ranjandeyranjan skin clinicshyderabad
20      soumyasarkarsoumya multi speciality clinichyde...
21                   rishankugoyalrkg eye centerhyderabad
                              ...                        
1731    kushangmodshivam children hospital and neonata...
1732              dhavalakhanichildhood hospitalahmedabad
1733    keval patelpateljk dental hospital orthodontic...
1734    jinal patelpateljk dental hospital orthodontic...
1735                      lavanyayasamsai clinichyderabad
Name: query, Length: 1715, dtype: object

In [22]:
from bs4 import BeautifulSoup
import urllib.request
import numpy as np
from googlesearch import search   


def extractReviews(df_not_missing):
    queries = df_not_missing['query'].values

    for i in tqdm(range(len(queries))):
        query = queries[i]
        query = "+".join(query.split(" "))
        url = 'https://google.com/search?q=dr+'+query
        

        
        for link in getLinkFromUrl(url):
            try:
                if "practo" in link:
                    # link wihtout ampersand
                    if "&" in link:
                        link_w = link.split("&")[0]
                    elif "%" in link:
                        link_w = link.split("&")[0]

                    request = urllib.request.Request(link_w)
                    request.add_header('User-Agent', 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36')

                
                
                    raw_response = urllib.request.urlopen(request).read()
                    # Read the repsonse as a utf-8 string
                    html = raw_response.decode("utf-8")

                    soup = BeautifulSoup(html, 'html.parser')

                    heading = soup.find("h1")
                    heading = heading.get_text()
                    print(heading)
                    print(link_w)

                    final_df['name'].append(heading)
                    final_df['link'].append(link_w)
            except:
                print("Couldn't get anything for : ", link_w)

                       
    return final_df

In [ ]:
final_df = extractReviews(df_not_missing)

  0%|                                        | 1/1715 [00:03<1:45:43,  3.70s/it]

Dr. Divya Varikallu
https://www.practo.com/hyderabad/doctor/divya-varikallu-general-practitioner


  0%|                                        | 2/1715 [00:07<1:39:45,  3.49s/it]

Shivam Clinic
https://www.practo.com/hyderabad/clinic/shivam-clinic-kphb


  0%|                                        | 3/1715 [00:09<1:23:36,  2.93s/it]

...

Dr. Reddy Sreenivasa Rao
https://www.practo.com/visakhapatnam/doctor/dr-reddy-sreenivasa-rao-gastroenterologist


  1%|▎                                      | 13/1715 [00:39<2:28:56,  5.25s/it]

Couldn't get anything for :  https://www.practo.com/visakhapatnam/clinic/sri-srinivasa-gastro-and-liver-centre-maharanipeta


  1%|▎                                      | 14/1715 [00:41<2:00:45,  4.26s/it]

Dr. J. k Vijay Kumar
https://www.practo.com/visakhapatnam/doctor/dr-j-k-vijay-kumar-urologist


In [ ]:
pd.DataFrame(final_df).to_csv("output_v1_practo.csv", index=False)

In [ ]:
final_df

In [ ]:
df_final = pd.DataFrame(final_df)

practo: name, address, working days, phone, speciality

In [ ]:
df_final

In [ ]:
df.columns

In [ ]:
pd.merge(df, df_final, how='outer').to_csv("full_output_30_aug_2022.csv", index=False)

In [ ]:
pd.read_csv("full_output_30_aug_2022.csv")

In [ ]:
df

### Processing for NA

In [ ]:
df = pd.read_csv("full_output_30_aug_2022.csv")
df.head()

In [ ]:
df_na = df[df['rating'].isna() == True]

In [ ]:
df_na = df_na[df_na['firstName'].isna()!=True].drop('clinicName', axis=1)

In [ ]:
df_na.columns = ['firstName', 'lastName', 'name', 'baseCity', 'GoogleNRs', 'googleRating']

In [ ]:
df_na